#Captura y Procesamiento de Video en Tiempo Real con Webcam

Este cuaderno está diseñado para ser utilizado en un entorno educativo, explicando paso a paso cómo capturar video desde una webcam en Google Colab y cómo aplicar operaciones básicas de visión por computadora usando OpenCV en tiempo real.

**Objetivos:**

* Entender la interacción entre Python y JavaScript en Google Colab para acceso a hardware (webcam).
* Implementar captura de video en tiempo real.
* Aplicar un ejemplo de procesamiento de imágenes (detección de rostros) con OpenCV.
* Comprender la estructura de un bucle de procesamiento de video.

1. **Importar Librerías Necesarias**

*   **cv2**: La biblioteca principal de OpenCV para procesamiento de imágenes y visión por computadora.
*   **IPython.display**: Utilizado para mostrar elementos HTML y JavaScript directamente en la salida de Colab. En particular, `display` y `Javascript` para ejecutar código en el navegador, y `Image` para mostrar los fotogramas de video.
*   **google.colab.output.eval_js**: Una función crucial de Colab que permite ejecutar una cadena de JavaScript y obtener su resultado de vuelta en Python.
*   **base64**: Para codificar y decodificar datos binarios (como imágenes) a/desde cadenas de texto.
*   **PIL (Pillow)** e **io**: Utilizados para manipular bytes de imagen y convertirlos entre diferentes formatos.
*   **time**: Para introducir pausas y medir el rendimiento (FPS).
*   **numpy**: Fundamental para el manejo de imágenes, ya que OpenCV representa las imágenes como arrays NumPy.

In [ ]:
import cv2
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import time
import numpy as np

2. **Funciones JavaScript para la Interacción con la Webcam**

Google Colab se ejecuta en servidores de Google, no directamente en tu máquina. Para acceder a tu webcam, necesitamos ejecutar código JavaScript en tu navegador. Esta celda define las funciones JavaScript necesarias para iniciar, detener y capturar fotogramas del stream de la cámara.

**Explicación de las funciones JavaScript:**

*   **`startStream()`**: Solicita permiso al navegador para acceder a la webcam (`navigator.mediaDevices.getUserMedia`). Si se concede, crea un elemento `<video>` en el DOM del navegador y comienza a reproducir el stream de la webcam en él. Es `async` porque `getUserMedia` es una operación asíncrona.
*   **`stopStream()`**: Detiene todas las pistas del stream de la webcam, pausa el video y elimina los elementos de video y canvas del DOM, liberando así la cámara.
*   **`captureFrame()`**: Toma una "instantánea" del fotograma actual del elemento `<video>`, lo dibuja en un `<canvas>` oculto y luego lo convierte a una cadena de datos Base64 (formato JPEG) para ser enviado a Python.
*   **`isStreamActive()`**: Una función auxiliar para verificar el estado del stream de video en el lado del navegador.

In [ ]:
def video_stream():
  """
  Define e inyecta el código JavaScript para manejar la transmisión de video
  desde la webcam del usuario.
  """
  js = Javascript('''
    var video;
    var stream;
    var captureCanvas; // Canvas para capturar frames
    var captureContext; // Contexto 2D del canvas

    // Función para iniciar la transmisión de video
    async function startStream() {
      try {
        stream = await navigator.mediaDevices.getUserMedia({video: true, audio: false});
        video = document.createElement('video');
        video.srcObject = stream;
        // Opcional: para ver el video directamente en el DOM, descomentar
        // video.style.maxWidth = '100%';
        // video.style.display = 'block';
        // document.body.appendChild(video);
        video.play();
        console.log("Stream de webcam iniciado en JS.");
      } catch (error) {
        console.log("Error al acceder a la webcam: ", error);
        alert("Por favor, permite el acceso a la cámara para usar esta función.");
      }
    }

    // Función para detener la transmisión de video
    function stopStream() {
      if (stream) {
        stream.getTracks().forEach(track => track.stop()); // Detiene todas las pistas
      }
      if (video) {
        video.pause();
        video.srcObject = null;
        video.remove(); // Elimina el elemento de video del DOM
      }
      if (captureCanvas) {
        captureCanvas.remove(); // Elimina el canvas
      }
      console.log("Webcam detenida en JS.");
    }

    // Función para capturar un único fotograma del video
    function captureFrame() {
      if (!video || !stream || !stream.active) {
        // Asegúrate de que el video esté reproduciéndose y el stream esté activo
        console.log("No hay stream de video activo para capturar.");
        return null;
      }
      if (!captureCanvas) {
        captureCanvas = document.createElement('canvas');
        // No lo adjuntamos al body, solo lo usamos internamente para la captura
      }
      // Ajustar el tamaño del canvas al tamaño actual del video
      captureCanvas.width = video.videoWidth;
      captureCanvas.height = video.videoHeight;
      captureContext = captureCanvas.getContext('2d');
      captureContext.drawImage(video, 0, 0, video.videoWidth, video.videoHeight);
      // Retorna el fotograma como Base64 JPEG con una calidad del 80%
      return captureCanvas.toDataURL('image/jpeg', 0.8);
    }

    // Función para verificar si el stream de video está activo
    function isStreamActive() {
        return (stream && stream.active && video && !video.paused);
    }

    // Llama a startStream() inmediatamente al ejecutar esta celda para iniciar el stream
    startStream();
  ''')
  display(js) # Ejecuta el JavaScript en el navegador

def video_frame():
  """
  Función Python que llama a la función JavaScript 'captureFrame()'
  para obtener un fotograma y lo devuelve como una cadena base64.
  """
  data = eval_js('captureFrame()')
  return data

def check_stream_active():
  """
  Función Python que verifica el estado del stream de JavaScript
  llamando a la función JavaScript 'isStreamActive()'.
  """
  return eval_js('isStreamActive()')

3. **Funciones Auxiliares de Python para Procesamiento de Imágenes**

Estas funciones actúan como "puentes" entre el formato de datos de JavaScript (Base64) y el formato que OpenCV (NumPy arrays) entiende, y viceversa. También incluyen la lógica para cargar el clasificador de rostros de OpenCV.

*   **`js_to_image(js_reply)`**: Toma la cadena Base64 que viene del navegador, la decodifica y la convierte en un array NumPy de OpenCV (BGR).
*   **`bbox_to_bytes(bbox_array)`**: Toma un array NumPy de OpenCV (BGR o RGB, se convierte a RGB para PIL) y lo convierte en un formato de bytes JPEG que `IPython.display.Image` puede mostrar.
*   **Carga del clasificador Haar Cascade**: El clasificador de rostros se carga una sola vez al principio. Esto es eficiente, ya que no se carga en cada fotograma.

In [ ]:
# --- Funciones Auxiliares de Python para Conversión de Imágenes ---

def js_to_image(js_reply):
  """
  Decodifica la cadena de datos base64 de una imagen recibida de JavaScript
  y la convierte a un formato de imagen OpenCV (numpy array BGR).
  """
  if js_reply is None:
    return None

  # El prefijo 'data:image/jpeg;base64,' debe ser removido
  image_bytes = b64decode(js_reply.split(',')[1])
  # Convertir bytes a una imagen PIL (Pillow)
  pil_img = PIL.Image.open(io.BytesIO(image_bytes))
  # Convertir la imagen PIL a un formato de OpenCV (NumPy array BGR)
  img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
  return img

def bbox_to_bytes(bbox_array):
  """
  Convierte una imagen NumPy array (OpenCV format) a bytes JPEG
  para que pueda ser mostrada por IPython.display.Image en Colab.
  """
  # Aseguramos que la imagen esté en formato RGB para PIL antes de guardar como JPEG
  pil_img = PIL.Image.fromarray(bbox_array, 'RGB')
  img_byte_arr = io.BytesIO()
  pil_img.save(img_byte_arr, format='JPEG')
  # Obtenemos los bytes del buffer
  bbox_bytes = img_byte_arr.getvalue()
  return bbox_bytes

# --- Carga del Clasificador de Rostros (se ejecuta una vez) ---
# Intentamos cargar el clasificador de Haar Cascades para detección de rostros.
# Esto se hace fuera del bucle principal para mayor eficiencia.
try:
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    if face_cascade.empty():
        # Si el archivo no se carga, lanzamos un error
        raise IOError('No se pudo cargar el archivo XML de Haar Cascade para rostros.')
    print("Clasificador de rostros cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el clasificador de rostros: {e}")
    face_cascade = None # Si la carga falla, la variable será None, y la detección se omitirá

4. **Lógica de Visión por Computadora (Función `process_frame`)**

Aquí es donde agregaremos y probaremos diferentes algoritmos de procesamiento de imágenes de OpenCV. La función `process_frame` toma un fotograma de la cámara y aplica las operaciones deseadas antes de devolver el resultado.

**Ejemplos incluidos:**

*   **Detección de Rostros**: Utiliza el clasificador de Haar Cascades cargado previamente para identificar rostros y dibujar rectángulos azules alrededor de ellos.
*   **Filtro de Escala de Grises y Canny**: (Comentado) Puedes descomentar estas líneas para ver cómo se aplican filtros de transformación de color y detección de bordes.
*   **Dibujar Texto**: Agrega el número de fotograma actual en la esquina superior izquierda de la imagen.

In [ ]:
# --- Lógica de Visión por Computadora (Función de Procesamiento de Fotogramas) ---

def process_frame(frame, frame_number=0):
  """
  Aplica diferentes efectos de visión por computadora al fotograma de entrada.
  Puedes activar/desactivar efectos comentando/descomentando las líneas correspondientes.

  Args:
    frame (numpy.ndarray): El fotograma de imagen en formato OpenCV (BGR).
    frame_number (int): El número del fotograma actual.

  Returns:
    numpy.ndarray: El fotograma procesado.
  """
  processed_frame = frame.copy() # Siempre trabajamos en una copia para no modificar el original

  # --- Ejemplo 1: Detección de Rostros ---
  # Solo se ejecuta si el clasificador de rostros se cargó correctamente
  if face_cascade is not None:
    # Convertir el fotograma a escala de grises para la detección de rostros
    gray_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2GRAY)
    # Detectar rostros en la imagen en escala de grises
    # scaleFactor: Parámetro que especifica cuánto se reduce la imagen en cada escala.
    # minNeighbors: Parámetro que especifica cuántos vecinos debe tener cada rectángulo candidato para retenerlo.
    # minSize: Tamaño mínimo posible del objeto. Objetos más pequeños se ignoran.
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=4, minSize=(30, 30))

    # Dibujar rectángulos alrededor de los rostros detectados
    for (x, y, w, h) in faces:
      cv2.rectangle(processed_frame, (x, y), (x+w, y+h), (255, 0, 0), 2) # Color azul (BGR), grosor 2

  # --- Ejemplo 2: Aplicar un filtro (Escala de Grises + Detección de Bordes Canny) ---
  # Descomenta las siguientes líneas para activar este efecto.
  # Nota: Si activas esto, el stream se verá en blanco y negro con los bordes.
  # gray_processed = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2GRAY)
  # edges = cv2.Canny(gray_processed, 100, 200) # Detecta bordes
  # processed_frame = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR) # Convierte de nuevo a BGR para visualización

  # --- Ejemplo 3: Dibujar texto en el fotograma ---
  text_to_display = f"Fotograma: {frame_number}"
  # Poner texto en la imagen: (imagen, texto, origen, fuente, escala, color, grosor, tipo de línea)
  cv2.putText(processed_frame, text_to_display, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

  # --- Ejemplo 4: Dibujar una forma ---
  # Descomenta esta línea para dibujar un círculo rojo en la esquina superior derecha
  # cv2.circle(processed_frame, (processed_frame.shape[1] - 50, 50), 30, (0, 0, 255), -1) # Círculo rojo relleno

  return processed_frame

5. **Bucle Principal de Captura y Visualización**

Esta es la celda central que orquesta todo. Llama a las funciones JavaScript para iniciar la cámara, luego entra en un bucle infinito donde captura fotogramas, los procesa con la lógica de visión por computadora y los muestra en la salida de Colab. Incluye mecanismos de seguridad para manejar interrupciones y asegurar que la cámara se apague correctamente.

**Flujo de ejecución:**

1.  **Inicio de la transmisión**: Llama a `video_stream()` para activar la webcam en el navegador.
2.  **Espera de activación**: Un bucle de espera verifica periódicamente (usando `check_stream_active()`) si la cámara está realmente activa antes de intentar capturar fotogramas. Esto previene el error "No se recibieron datos del fotograma".
3.  **Bucle de captura**:
    *   `js_reply = video_frame()`: Solicita el fotograma actual del navegador.
    *   `img = js_to_image(js_reply)`: Convierte el fotograma Base64 a un array NumPy de OpenCV.
    *   `processed_img = process_frame(img, frame_number=frame_count)`: Pasa el fotograma a nuestra función de procesamiento de visión por computadora.
    *   `jpeg_bytes = bbox_to_bytes(processed_img_rgb)`: Convierte el fotograma procesado de nuevo a bytes para mostrarlo.
    *   `display_handle.update(Image(data=jpeg_bytes))`: Actualiza la imagen mostrada en la salida de Colab.
    *   **Cálculo de FPS**: Opcionalmente, se muestra la tasa de fotogramas por segundo para monitorear el rendimiento.
    *   `time.sleep(0.01)`: Una pequeña pausa para evitar saturar la CPU y permitir que la interfaz de Colab se actualice fluidamente.
4.  **Manejo de errores y finalización (try...except...finally)**:
    *   El bucle puede ser detenido por `KeyboardInterrupt` (cuando el usuario presiona el botón de "stop" en Colab).
    *   El bloque `finally` asegura que la función `stopStream()` de JavaScript sea llamada para apagar la webcam, independientemente de cómo termine el bucle.

In [ ]:
# --- Bucle Principal de Captura y Visualización ---

def start_webcam_capture():
  """
  Inicia la captura de video en tiempo real desde la webcam,
  procesa cada fotograma y lo muestra en Colab.
  """
  print("Iniciando transmisión de webcam... Por favor, permite el acceso a la cámara.")
  video_stream() # Ejecuta el JavaScript para iniciar la cámara en el navegador

  # Esperar a que el stream de la cámara esté activo en el navegador
  print("Esperando que el stream de la cámara esté activo", end="")
  max_wait_time = 15 # Tiempo máximo de espera en segundos
  wait_interval = 0.5 # Intervalo de verificación en segundos
  start_wait = time.time()

  # Bucle para verificar si el stream de la cámara está activo
  while not check_stream_active() and (time.time() - start_wait) < max_wait_time:
      print(".", end="") # Muestra puntos para indicar que está esperando
      time.sleep(wait_interval)
  print("\n") # Salto de línea después de los puntos de espera

  # Si la cámara no se activó a tiempo, informamos y salimos
  if not check_stream_active():
      print("Error: El stream de la cámara no se activó a tiempo.")
      print("Asegúrate de haber dado los permisos necesarios y de que la cámara no esté siendo usada por otra aplicación.")
      eval_js('stopStream();') # Intentamos apagar la cámara por si acaso
      return # Salir de la función si la cámara no está lista

  # display_handle es el objeto que se actualizará con cada nuevo fotograma en Colab
  display_handle = display(None, display_id=True)
  frame_count = 0
  start_time = time.time()

  try:
    while True: # Bucle infinito para capturar y procesar fotogramas continuamente
      # Captura un fotograma de la webcam a través de JavaScript
      js_reply = video_frame()

      # Si no se reciben datos, el stream se ha detenido o ha habido un problema
      if js_reply is None:
        print("No se recibieron datos del fotograma. La transmisión puede haberse detenido o fallado.")
        break # Salir del bucle principal

      # Convierte el fotograma de JavaScript (cadena base64) a una imagen OpenCV (NumPy array)
      img = js_to_image(js_reply)

      # --- LLAMADA A LA LÓGICA DE VISIÓN POR COMPUTADURA ---
      # Aquí es donde aplicamos nuestros algoritmos de procesamiento al fotograma
      processed_img = process_frame(img, frame_number=frame_count)

      # Convierte la imagen procesada (NumPy array) a bytes JPEG para la visualización en Colab
      # Es importante convertir a RGB antes de la conversión a bytes si OpenCV está en BGR
      processed_img_rgb = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)
      jpeg_bytes = bbox_to_bytes(processed_img_rgb)

      # Actualiza la visualización del fotograma en la salida de Colab
      display_handle.update(Image(data=jpeg_bytes))

      frame_count += 1
      # Opcional: Calcula y muestra los FPS (Fotogramas Por Segundo) cada 30 fotogramas
      if frame_count % 30 == 0:
          end_time = time.time()
          fps = frame_count / (end_time - start_time)
          print(f"FPS: {fps:.2f}")
          frame_count = 0 # Resetea el contador para el próximo cálculo de FPS
          start_time = time.time() # Resetea el tiempo de inicio

      # Pequeña pausa para evitar sobrecargar la CPU y permitir la actualización de la interfaz
      time.sleep(0.01)

  except KeyboardInterrupt:
    # Se captura cuando el usuario presiona el botón de "Stop" en Colab
    print("\nCaptura de video detenida por interrupción del usuario.")
  except Exception as e:
    # Captura cualquier otro error inesperado
    print(f"\nOcurrió un error inesperado durante la captura: {e}")
  finally:
    # Este bloque se ejecuta siempre al salir del bucle (sea por break, KeyboardInterrupt o error)
    print("Limpiando y deteniendo la webcam...")
    eval_js('stopStream();') # Llama al JavaScript para detener la transmisión de la webcam
    # Limpia la salida visible en Colab para eliminar el video
    display_handle.update(Javascript('google.colab.output.clear()'))
    print("Proceso de captura finalizado.")

6. **Ejecutar el Cuaderno**

Finalmente, esta es la celda que llamarás para iniciar todo el proceso.

In [ ]:
# --- Ejecutar el Cuaderno ---
if __name__ == '__main__':
  start_webcam_capture()
  print("\n--- Proceso Completado ---")
  print("Para iniciar una nueva captura, simplemente ejecuta la celda anterior (Bucle Principal) nuevamente.")
  print("Si la cámara no se apaga completamente (raro, pero posible), puedes ejecutar la siguiente celda individualmente:")
  print("eval_js('stopStream();')")

7. **Detener la Transmisión de la Webcam (Manual - Opcional)**

En muy raras ocasiones, si la ejecución del bucle principal se detuvo de forma abrupta y la cámara no se apagó, puedes ejecutar esta celda para forzar el cierre del stream de la webcam.

In [ ]:
# --- Detener la Transmisión de la Webcam (Manual - Opcional) ---
from google.colab.output import eval_js

print("Intentando detener la webcam manualmente...")
eval_js('stopStream();')
print("Webcam detenida manualmente.")

8. **Prueba de Acceso Directo a la Webcam (JavaScript)**

Este código es una prueba simple para verificar si se puede acceder a la webcam directamente utilizando JavaScript en el navegador. Intenta obtener el stream de video, crear un elemento de video y reproducirlo en la salida de Colab. También incluye un manejo básico de errores si el acceso a la cámara falla.

In [ ]:
from IPython.display import display, Javascript

js_test = Javascript('''
  navigator.mediaDevices.getUserMedia({video: true, audio: false})
    .then(function(stream) {
      var video = document.createElement('video');
      video.srcObject = stream;
      video.style.maxWidth = '100%'; // Para que el video se ajuste
      video.style.display = 'block'; // Asegura que se muestre
      document.body.appendChild(video);
      video.play();
      console.log('Cámara activada en JavaScript. Buscando video...');
    })
    .catch(function(error) {
      console.log('Error al acceder a la cámara:', error);
      alert('Error al acceder a la cámara. Revisa los permisos: ' + error.name);
    });
''')
display(js_test)

<IPython.core.display.Javascript object>